In [3]:
#Librerias 
import pandas as pd
import numpy as np
import pyodbc
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPClassifier 

import matplotlib
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')


In [4]:
#funciones
def convertir_tag_rendimiento(value):
    if value == 0:
        return 'EXITOSO'
    elif value == 1:
        return 'NO EXITOSO'
    else:
        return 'ERROR'

## 01 Seleccion de datos

In [5]:
conn = pyodbc.connect(DRIVER='{ODBC Driver 17 for SQL Server}', SERVER='MED000008696', DATABASE='Difods_Analytics', Trusted_Connection='yes')
cursor = conn.cursor()

datos_estructurados_exitoso = pd.read_sql_query("SELECT TOP(100000) * FROM [Difods_Analytics].dbo.DATA_TRAIN_RNA_FINAL WHERE RENDIMIENTO='EXITOSO'", conn)
datos_estructurados_no_exitoso = pd.read_sql_query("SELECT TOP(100000) * FROM [Difods_Analytics].dbo.DATA_TRAIN_RNA_FINAL WHERE RENDIMIENTO='NO EXITOSO'", conn)

In [6]:
datos_estructurados_total = datos_estructurados_exitoso.append(datos_estructurados_no_exitoso, ignore_index=True)

In [7]:
datos_estructurados_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 47 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   RENDIMIENTO                      200000 non-null  object 
 1   Descripcion_Cargo                200000 non-null  object 
 2   Descripcion_Subtipo_trabajador   200000 non-null  object 
 3   escala                           200000 non-null  object 
 4   Jornada_Laboral                  200000 non-null  int64  
 5   condicion_laboral                200000 non-null  object 
 6   D_NIV_MOD                        200000 non-null  object 
 7   D_COD_CAR                        200000 non-null  object 
 8   DAREACENSO                       200000 non-null  object 
 9   D_REGION                         200000 non-null  object 
 10  nivel                            200000 non-null  object 
 11  JEC                              200000 non-null  bool   
 12  EI

In [8]:
datos_estructurados_total.head(10)

,RENDIMIENTO,Descripcion_Cargo,Descripcion_Subtipo_trabajador,escala,Jornada_Laboral,condicion_laboral,D_NIV_MOD,D_COD_CAR,DAREACENSO,D_REGION,...,CHOICE,PAGE,ASSIGN,URL,PRUEBA_ENTRADA,FINALGRADE,PUNTAJE_PUN,CONDICION_PUN,FORO_PRESENTACION,PUNTUACION_FORO
0,EXITOSO,PROFESOR,DOCENTE,4,30,NOMBRADO,Inicial - Jardín,No aplica,Urbana,DRE LIMA METROPOLITANA,...,0,1,0,10,1,16.0,0.0,Otro,0,0.0
1,EXITOSO,PROFESOR,DOCENTE,1,30,CONTRATADO,Inicial - Jardín,No aplica,Urbana,DRE AREQUIPA,...,0,1,0,10,1,14.0,135.5,No clasificado,0,0.0
2,EXITOSO,PROFESOR,DOCENTE,2,30,NOMBRADO,Secundaria,No aplica,Rural,DRE LA LIBERTAD,...,0,0,0,10,1,14.0,0.0,Otro,0,0.0
3,EXITOSO,PROFESOR,DOCENTE,5,30,NOMBRADO,Secundaria,No aplica,Rural,DRE CAJAMARCA,...,0,0,0,10,1,20.0,0.0,Otro,0,0.0
4,EXITOSO,PROFESOR,DOCENTE,1,20,CONTRATADO,Secundaria,No aplica,Rural,DRE LIMA PROVINCIAS,...,0,1,0,8,1,6.0,84.0,No clasificado,0,0.0
5,EXITOSO,PROFESOR,DOCENTE,1,30,CONTRATADO,Inicial - Jardín,No aplica,Rural,DRE ANCASH,...,0,1,0,10,1,6.0,86.0,No clasificado,0,0.0
6,EXITOSO,PROFESOR,DOCENTE,1,30,CONTRATADO,Inicial - Jardín,No aplica,Urbana,DRE LIMA METROPOLITANA,...,0,1,0,10,1,20.0,74.0,No clasificado,0,0.0
7,EXITOSO,PROFESOR,DOCENTE,1,30,NOMBRADO,Básica Alternativa-Inicial e Intermedio,No aplica,Urbana,DRE CUSCO,...,0,1,0,10,1,6.0,135.5,Clasificado,0,0.0
8,EXITOSO,PROFESOR,DOCENTE,1,30,CONTRATADO,Primaria,Polidocente Completo,Rural,DRE PUNO,...,0,1,0,8,1,10.0,106.0,No clasificado,0,0.0
9,EXITOSO,PROFESOR,DOCENTE,1,30,CONTRATADO,Secundaria,No aplica,Urbana,DRE TACNA,...,0,1,0,8,1,10.0,136.5,No clasificado,0,0.0


## 02 Pre Procesamiento

In [9]:
le_rendimiento = LabelEncoder()
datos_estructurados_total['RENDIMIENTO'] = le_rendimiento.fit_transform(datos_estructurados_total['RENDIMIENTO'])

le_cargo = LabelEncoder()
datos_estructurados_total['Descripcion_Cargo'] = le_cargo.fit_transform(datos_estructurados_total['Descripcion_Cargo'])

le_subtipo_trabajador = LabelEncoder()
datos_estructurados_total['Descripcion_Subtipo_trabajador'] = le_subtipo_trabajador.fit_transform(datos_estructurados_total['Descripcion_Subtipo_trabajador'])

le_condicion_laboral = LabelEncoder()
datos_estructurados_total['condicion_laboral'] = le_condicion_laboral.fit_transform(datos_estructurados_total['condicion_laboral'])

le_dniv = LabelEncoder()
datos_estructurados_total['D_NIV_MOD'] = le_dniv.fit_transform(datos_estructurados_total['D_NIV_MOD'])

le_dcodcar = LabelEncoder()
datos_estructurados_total['D_COD_CAR'] = le_dcodcar.fit_transform(datos_estructurados_total['D_COD_CAR'])

le_dareacenso = LabelEncoder()
datos_estructurados_total['DAREACENSO'] = le_dareacenso.fit_transform(datos_estructurados_total['DAREACENSO'])

le_region = LabelEncoder()
datos_estructurados_total['D_REGION'] = le_region.fit_transform(datos_estructurados_total['D_REGION'])

le_nivel = LabelEncoder()
datos_estructurados_total['nivel'] = le_nivel.fit_transform(datos_estructurados_total['nivel'])

le_atencion_EIB = LabelEncoder()
datos_estructurados_total['Forma de atención EIB'] = le_atencion_EIB.fit_transform(datos_estructurados_total['Forma de atención EIB'])

#le_lengua = LabelEncoder()
#datos_estructurados_total['Nombre de lengua originaria 1 - 2019'] = le_lengua.fit_transform(datos_estructurados_total['Nombre de lengua originaria 1 - 2019'])

le_lengua2 = LabelEncoder()
datos_estructurados_total['Lengua Originaria sin variante'] = le_lengua2.fit_transform(datos_estructurados_total['Lengua Originaria sin variante'])

le_situacion_laboral = LabelEncoder()
datos_estructurados_total['Situación_Laboral'] = le_situacion_laboral.fit_transform(datos_estructurados_total['Situación_Laboral'])

le_sexo = LabelEncoder()
datos_estructurados_total['sexo'] = le_sexo.fit_transform(datos_estructurados_total['sexo'])

#le_plaza = LabelEncoder()
#datos_estructurados_total['Estado_plaza'] = le_plaza.fit_transform(datos_estructurados_total['Estado_plaza'])

le_forma = LabelEncoder()
datos_estructurados_total['d_forma'] = le_forma.fit_transform(datos_estructurados_total['d_forma'])

le_gestion = LabelEncoder()
datos_estructurados_total['d_gestion'] = le_gestion.fit_transform(datos_estructurados_total['d_gestion'])

le_ges_dep = LabelEncoder()
datos_estructurados_total['d_ges_dep'] = le_ges_dep.fit_transform(datos_estructurados_total['d_ges_dep'])

le_ruralidad = LabelEncoder()
datos_estructurados_total['ruralidad_2021'] = le_ruralidad.fit_transform(datos_estructurados_total['ruralidad_2021'])

le_calidad_cobertura1 = LabelEncoder()
datos_estructurados_total['calidad_cobertura_2t2020'] = le_calidad_cobertura1.fit_transform(datos_estructurados_total['calidad_cobertura_2t2020'])

le_calidad_cobertura2 = LabelEncoder()
datos_estructurados_total['calidad_cobertura_internet_fijo'] = le_calidad_cobertura2.fit_transform(datos_estructurados_total['calidad_cobertura_internet_fijo'])

le_condicion_pun = LabelEncoder()
datos_estructurados_total['CONDICION_PUN'] = le_condicion_pun.fit_transform(datos_estructurados_total['CONDICION_PUN'])


## 03 Transformación

In [ ]:
le_rendimiento.inverse_transform([0, 1])

In [10]:
### Estandarización de datos
scaler_jl = MinMaxScaler()
datos_estructurados_total['Jornada_Laboral'] = scaler_jl.fit_transform(np.array(datos_estructurados_total['Jornada_Laboral']).reshape(-1, 1))

scaler_ndl = MinMaxScaler()
datos_estructurados_total['Numero de limitaciones'] = scaler_ndl.fit_transform(np.array(datos_estructurados_total['Numero de limitaciones']).reshape(-1, 1))

scaler_edad = MinMaxScaler()
datos_estructurados_total['edad'] = scaler_edad.fit_transform(np.array(datos_estructurados_total['edad']).reshape(-1, 1))

scaler1 = MinMaxScaler()
datos_estructurados_total['FEEDBACK'] = scaler1.fit_transform(np.array(datos_estructurados_total['FEEDBACK']).reshape(-1, 1))

scaler2 = MinMaxScaler()
datos_estructurados_total['FORUM'] = scaler2.fit_transform(np.array(datos_estructurados_total['FORUM']).reshape(-1, 1))

scaler3 = MinMaxScaler()
datos_estructurados_total['RESOURCE'] = scaler3.fit_transform(np.array(datos_estructurados_total['RESOURCE']).reshape(-1, 1))

scaler4 = MinMaxScaler()
datos_estructurados_total['LABEL'] = scaler4.fit_transform(np.array(datos_estructurados_total['LABEL']).reshape(-1, 1))

scaler5 = MinMaxScaler()
datos_estructurados_total['QUIZ'] = scaler5.fit_transform(np.array(datos_estructurados_total['QUIZ']).reshape(-1, 1))

scaler6 = MinMaxScaler()
datos_estructurados_total['CHOICE'] = scaler6.fit_transform(np.array(datos_estructurados_total['CHOICE']).reshape(-1, 1))

scaler7 = MinMaxScaler()
datos_estructurados_total['PAGE'] = scaler7.fit_transform(np.array(datos_estructurados_total['PAGE']).reshape(-1, 1))

scaler8 = MinMaxScaler()
datos_estructurados_total['ASSIGN'] = scaler8.fit_transform(np.array(datos_estructurados_total['ASSIGN']).reshape(-1, 1))

scaler9 = MinMaxScaler()
datos_estructurados_total['URL'] = scaler9.fit_transform(np.array(datos_estructurados_total['URL']).reshape(-1, 1))

scaler10 = MinMaxScaler()
datos_estructurados_total['FINALGRADE'] = scaler10.fit_transform(np.array(datos_estructurados_total['FINALGRADE']).reshape(-1, 1))

scaler11 = MinMaxScaler()
datos_estructurados_total['PUNTAJE_PUN'] = scaler11.fit_transform(np.array(datos_estructurados_total['PUNTAJE_PUN']).reshape(-1, 1))


In [11]:
datos_estructurados_total.head(10)

,RENDIMIENTO,Descripcion_Cargo,Descripcion_Subtipo_trabajador,escala,Jornada_Laboral,condicion_laboral,D_NIV_MOD,D_COD_CAR,DAREACENSO,D_REGION,...,CHOICE,PAGE,ASSIGN,URL,PRUEBA_ENTRADA,FINALGRADE,PUNTAJE_PUN,CONDICION_PUN,FORO_PRESENTACION,PUNTUACION_FORO
0,0,2,1,4,0.729730,1,7,0,1,14,...,0.0,0.111111,0.0,0.192308,1,0.8,0.000000,2,0,0.0
1,0,2,1,1,0.729730,0,7,0,1,3,...,0.0,0.111111,0.0,0.192308,1,0.7,0.700258,1,0,0.0
2,0,2,1,2,0.729730,1,9,0,0,12,...,0.0,0.000000,0.0,0.192308,1,0.7,0.000000,2,0,0.0
3,0,2,1,5,0.729730,1,9,0,0,5,...,0.0,0.000000,0.0,0.192308,1,1.0,0.000000,2,0,0.0
4,0,2,1,1,0.459459,0,9,0,0,15,...,0.0,0.111111,0.0,0.153846,1,0.3,0.434109,1,0,0.0
5,0,2,1,1,0.729730,0,7,0,0,1,...,0.0,0.111111,0.0,0.192308,1,0.3,0.444444,1,0,0.0
6,0,2,1,1,0.729730,0,7,0,1,14,...,0.0,0.111111,0.0,0.192308,1,1.0,0.382429,1,0,0.0
7,0,2,1,1,0.729730,1,1,0,1,7,...,0.0,0.111111,0.0,0.192308,1,0.3,0.700258,0,0,0.0
8,0,2,1,1,0.729730,0,8,2,0,21,...,0.0,0.111111,0.0,0.153846,1,0.5,0.547804,1,0,0.0
9,0,2,1,1,0.729730,0,9,0,1,23,...,0.0,0.111111,0.0,0.153846,1,0.5,0.705426,1,0,0.0


## 04 Mineria de datos

In [12]:
y = datos_estructurados_total['RENDIMIENTO']
X = datos_estructurados_total.drop(columns=['RENDIMIENTO'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [13]:
### Implementación del modelo
modelo_rna1 = MLPClassifier((100, 100), random_state=1, max_iter=300).fit(X_train, y_train)

y_pred_train = modelo_rna1.predict(X_train)
y_pred_test = modelo_rna1.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

prec_train = precision_score(y_train, y_pred_train)
prec_test = precision_score(y_test, y_pred_test)

rec_train = recall_score(y_train, y_pred_train)
rec_test = recall_score(y_test, y_pred_test)

f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

cm_train = confusion_matrix(y_train, y_pred_train)
cm_test = confusion_matrix(y_test, y_pred_test)

df_resultados = pd.DataFrame([[acc_train, acc_test], [prec_train, prec_test], [rec_train, rec_test], [f1_train, f1_test]], columns=['Entrenamiento','Prueba'], index=['Accuracy', 'Precision', 'Recall', 'F1-score'])


In [14]:
print('Matriz de confusión del dataset de entrenamiento:')
print(cm_train)
print('Matriz de confusión del dataset de prueba:')
print(cm_test)

print('Tabla de resultados:')
print(df_resultados)

Matriz de confusión del dataset de entrenamiento:
[[79183   817]
 [   24 79976]]
Matriz de confusión del dataset de prueba:
[[19774   226]
 [   15 19985]]
Tabla de resultados:
           Entrenamiento    Prueba
Accuracy        0.994744  0.993975
Precision       0.989888  0.988818
Recall          0.999700  0.999250
F1-score        0.994770  0.994007


In [15]:
label_rendimiento = ['EXITOSO', 'NO EXITOSO']